In [2]:
import pandas as pd
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import warnings
warnings.simplefilter('ignore')

In [3]:
country_virus=pd.read_csv("country_by_date.csv")
country_age = pd.read_csv("age_ratio.csv")
country_age = country_age.rename(columns={'Entity': 'Country_Region'})
country_age = country_age.loc[(country_age['Year'] == 2017)]
country_age = country_age[['Country_Region','Age dependency ratio (% of working-age population) (% of working-age population)']]

In [22]:
country_virus

,Unnamed: 0,Country_Region,Date,ConfirmedCases
0,0,Afghanistan,2020-01-22,0.0
1,5456,Afghanistan,2020-02-12,0.0
2,5458,Afghanistan,2020-02-10,0.0
3,5459,Afghanistan,2020-02-09,0.0
4,5460,Afghanistan,2020-02-08,0.0
...,...,...,...,...
13315,5415,Zimbabwe,2020-03-05,0.0
13316,5416,Zimbabwe,2020-03-04,0.0
13317,5427,Zimbabwe,2020-02-15,0.0
13318,5429,Zimbabwe,2020-02-17,0.0


In [4]:
#create a new table to get the country demographics data
countries = country_virus['Country_Region'].unique()
countries = pd.DataFrame (countries)
countries = countries.rename(columns={0: 'Country_Region'})

new_table = pd.merge(countries, country_age, on='Country_Region', how='left')
new_table = new_table.rename(columns={'Age dependency ratio (% of working-age population) (% of working-age population)':'Age Dependency Ratio (% of working-age population)'})
new_table

,Country_Region,Age Dependency Ratio (% of working-age population)
0,Afghanistan,84.587278
1,Albania,44.059506
2,Algeria,55.028217
3,Andorra,NaN
4,Angola,96.898518
...,...,...
175,Venezuela,51.996675
176,Vietnam,43.288199
177,West Bank and Gaza,NaN
178,Zambia,89.613257


In [5]:
physician = pd.read_csv("physicians_per_capita.csv")
physician = physician[['Country Name','Most_Recent']]
physician = physician.rename(columns={'Country Name': 'Country_Region'})
physician = physician.rename(columns={'Most_Recent': 'Physician_per_thousand'})
new_table = pd.merge(new_table, physician, on='Country_Region', how='left')
new_table

,Country_Region,Age Dependency Ratio (% of working-age population),Physician_per_thousand
0,Afghanistan,84.587278,0.3039
1,Albania,44.059506,1.6500
2,Algeria,55.028217,1.8300
3,Andorra,NaN,4.0000
4,Angola,96.898518,0.2149
...,...,...,...
175,Venezuela,51.996675,1.9400
176,Vietnam,43.288199,1.1800
177,West Bank and Gaza,NaN,0.8370
178,Zambia,89.613257,0.1632


In [28]:
other_demo = pd.read_csv("other_demo_info_edit.csv")

In [29]:
import datetime
ts = pd.Timestamp(year = 2011,  month = 11, day = 21,  
                  hour = 10, second = 49, tz = 'US/Pacific')
other_demo = other_demo[['country','quarantine','hospibed','pop']]
other_demo['quarantine'] = pd.to_datetime(other_demo['quarantine'])
other_demo['quarantine time'] =  ts.today() - other_demo['quarantine']
other_demo = other_demo.rename(columns={'country': 'Country_Region'})
#other_demo['quarantine time'] = other_demo.groupby(['Country_Region'])['quarantine'].max()


In [30]:
other_demo

,Country_Region,quarantine,hospibed,pop,quarantine time
0,Afghanistan,NaT,0.5,"38,928,346",NaT
1,Albania,NaT,2.9,"2,877,797",NaT
2,Algeria,NaT,1.9,"43,851,044",NaT
3,Andorra,NaT,2.5,"77,265",NaT
4,Antigua and Barbuda,NaT,3.8,"97,929",NaT
...,...,...,...,...,...
174,Venezuela,NaT,0.8,"28,435,940",NaT
175,Vietnam,2020-02-13,2.6,"97,338,579",53 days 16:06:38.356111
176,Zambia,NaT,2.0,"18,383,955",NaT
177,Zimbabwe,NaT,1.7,"14,862,924",NaT


In [31]:
other_demo

,Country_Region,quarantine,hospibed,pop,quarantine time
0,Afghanistan,NaT,0.5,"38,928,346",NaT
1,Albania,NaT,2.9,"2,877,797",NaT
2,Algeria,NaT,1.9,"43,851,044",NaT
3,Andorra,NaT,2.5,"77,265",NaT
4,Antigua and Barbuda,NaT,3.8,"97,929",NaT
...,...,...,...,...,...
174,Venezuela,NaT,0.8,"28,435,940",NaT
175,Vietnam,2020-02-13,2.6,"97,338,579",53 days 16:06:38.356111
176,Zambia,NaT,2.0,"18,383,955",NaT
177,Zimbabwe,NaT,1.7,"14,862,924",NaT


In [64]:
country_by_date=pd.read_csv("join-table/country_by_date.csv")
country_by_date
country_demographics_date = country_by_date.merge(country_age, on=['Country_Region'], how='left')
country_demographics_date = country_demographics_date.merge(physician, on=['Country_Region'], how='left')
country_demographics_date = country_demographics_date.merge(other_demo, on=['Country_Region'], how='left')
country_demographics_date

,Unnamed: 0,i,Country_Region,Date,ConfirmedCases,Population Density (per sq. km),HDI,Testing By Date,Total Testing,Age dependency ratio (% of working-age population) (% of working-age population),Physician_per_thousand,quarantine,hospibed,pop,quarantine time
0,0,0,Afghanistan,2020-01-22,0,56.937760,0.498,NaN,NaN,84.587278,0.3039,NaT,0.5,"38,928,346",NaT
1,1,5476,Afghanistan,2020-01-23,0,56.937760,0.498,NaN,NaN,84.587278,0.3039,NaT,0.5,"38,928,346",NaT
2,2,5475,Afghanistan,2020-01-24,0,56.937760,0.498,NaN,NaN,84.587278,0.3039,NaT,0.5,"38,928,346",NaT
3,3,5474,Afghanistan,2020-01-25,0,56.937760,0.498,NaN,NaN,84.587278,0.3039,NaT,0.5,"38,928,346",NaT
4,4,5473,Afghanistan,2020-01-26,0,56.937760,0.498,NaN,NaN,84.587278,0.3039,NaT,0.5,"38,928,346",NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13315,13315,9279,Zimbabwe,2020-03-31,8,37.324591,0.535,NaN,NaN,78.656894,0.2020,NaT,1.7,"14,862,924",NaT
13316,13316,9277,Zimbabwe,2020-04-01,8,37.324591,0.535,NaN,NaN,78.656894,0.2020,NaT,1.7,"14,862,924",NaT
13317,13317,9372,Zimbabwe,2020-04-02,9,37.324591,0.535,NaN,NaN,78.656894,0.2020,NaT,1.7,"14,862,924",NaT
13318,13318,9443,Zimbabwe,2020-04-03,9,37.324591,0.535,NaN,NaN,78.656894,0.2020,NaT,1.7,"14,862,924",NaT


In [ ]:
country_demographics_date.to_csv("country_demographics_by.csv")

In [68]:
country_demographics = country_demographics_date[['Country_Region','Population Density (per sq. km)','HDI',
                                                  'Total Testing','quarantine','hospibed','pop','quarantine time','Age dependency ratio (% of working-age population) (% of working-age population)',
                                                 'Physician_per_thousand']]

In [70]:
country_demographics.drop_duplicates()

,Country_Region,Population Density (per sq. km),HDI,Total Testing,quarantine,hospibed,pop,quarantine time,Age dependency ratio (% of working-age population) (% of working-age population),Physician_per_thousand
0,Afghanistan,56.937760,0.498,NaN,NaT,0.5,"38,928,346",NaT,84.587278,0.3039
74,Albania,104.612263,0.785,NaN,NaT,2.9,"2,877,797",NaT,44.059506,1.6500
148,Algeria,17.730075,0.754,NaN,NaT,1.9,"43,851,044",NaT,55.028217,1.8300
222,Andorra,163.842553,0.858,NaN,NaT,2.5,"77,265",NaT,NaN,4.0000
296,Angola,24.713052,0.581,NaN,NaT,NaN,NaN,NaT,96.898518,0.2149
...,...,...,...,...,...,...,...,...,...,...
12950,Venezuela,32.730792,0.761,NaN,NaT,0.8,"28,435,940",NaT,51.996675,1.9400
13024,Vietnam,308.125246,0.694,15637.0,2020-02-13,2.6,"97,338,579",53 days 16:06:38.356111,43.288199,1.1800
13098,West Bank and Gaza,758.984551,NaN,NaN,NaT,NaN,NaN,NaT,NaN,0.8370
13172,Zambia,23.341479,0.588,NaN,NaT,2.0,"18,383,955",NaT,89.613257,0.1632


In [71]:
country_demographics.to_csv("country_demographics.csv")